In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install nltk tensorflow-hub

In [2]:
import preprocessor  # this will download some data

[nltk_data] Downloading package punkt to /Users/svitko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/svitko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/svitko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import preprocessor
import pandas as pd
import pickle
import os

In [4]:
def maybe_process(store_file="data.pkl", force=False):
    if force or not os.path.exists(store_file):
        data = preprocessor.process_dir("dump/issues")
        with open(store_file, 'wb') as data_file:
            pickle.dump(data, data_file)
    else:
        with open(store_file, 'rb') as data_file:
            data = pickle.load(data_file)
    return data

In [5]:
data = maybe_process()

In [7]:
df = pd.DataFrame(data)
df = df.fillna("Unknown")

In [ ]:
user_vocabulary = pd.concat([df["assignee"], df["reporter"]]).unique()
assignee_vocabulary = df["assignee"].unique()

In [ ]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)
with open("assignee_vocabulary.pkl", 'wb') as assignee_assignee_vocabulary_file:
    pickle.dump(assignee_vocabulary, assignee_assignee_vocabulary_file)

In [ ]:
vocabulary_map = {"Unknown": 0}
count = 0
for el in user_vocabulary:
    count += 1
    vocabulary_map[el] = count

In [ ]:
with open("vocabulary.pkl", 'wb') as vocabulary_file:
    pickle.dump(user_vocabulary, vocabulary_file)

with open("vocabulary_map.pkl", 'wb') as vocabulary_file:
    pickle.dump(vocabulary_map, vocabulary_file)

Start tensorflow stuff here

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import shutil
import numpy as np

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Now, split the data into two parts -- training and evaluation.

In [ ]:
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]

In [ ]:
def add_more_features(df):
    # TODO: Add more features to the dataframe
    df["summary_clean"] = df["summary"].apply(lambda x: " ".join(preprocessor.process_text(x)))
    df["description_clean"] = df["description"].apply(lambda x: " ".join(preprocessor.process_text(x)))
    return df

In [ ]:
# Create pandas input function
def make_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x = add_more_features(df),
#         y = df["assignee"].fillna('Unknown').apply(lambda x: vocabulary_map[x]),
        y = df["assignee"],
        batch_size = 128,
        num_epochs = num_epochs,
        shuffle = True,
        queue_capacity = 1000,
        num_threads = 1
      )

In [ ]:
# Define your feature columns
def create_feature_cols():
    return [
#         tf.feature_column.categorical_column_with_vocabulary_list("reporter", 
#                                                                   vocabulary_list=user_vocabulary.tolist(), 
#                                                                   default_value=0)


        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                "reporter", vocabulary_list=user_vocabulary.tolist(), 
                default_value=0),
            1500,
        ),
        
#         hub.text_embedding_column(
#             key="description_clean", 
#             module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
#             trainable=True,
#         ),
        
#         hub.text_embedding_column(
#             key="summary_clean", 
#             module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
#             trainable=True,
#         ),
        
        hub.text_embedding_column(
            key="description_clean", 
            module_spec="https://tfhub.dev/google/Wiki-words-250-with-normalization/1",
            trainable=True,
        ),
        
        hub.text_embedding_column(
            key="summary_clean", 
            module_spec="https://tfhub.dev/google/Wiki-words-250-with-normalization/1",
            trainable=True,
        ),
        
    ]

In [ ]:
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.DNNClassifier(
        model_dir = output_dir,
        feature_columns = create_feature_cols(),
        hidden_units=[200, 100, 50],
        n_classes=len(assignee_vocabulary.tolist()),
        label_vocabulary = assignee_vocabulary.tolist(),
    )
    
#     estimator = tf.estimator.LinearClassifier(
#         model_dir = output_dir, 
#         feature_columns = create_feature_cols(),
#         n_classes=len(user_vocabulary.tolist()),
#         label_vocabulary = user_vocabulary.tolist(),
#     )
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(traindf, None),
                                        max_steps = num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(evaldf, 1), 
                                      steps = None, 
                                      start_delay_secs = 1, # start evaluating after N seconds, 
                                      throttle_secs = 10)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [ ]:
OUTDIR = './trained_model'

In [ ]:
# Run the model
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, 2000)